In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import glob as glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats

In [3]:
df=pd.read_csv('./gdrive/My Drive/Deanonymization/all_three.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427625 entries, 0 to 427624
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        427625 non-null  int64  
 1   balance           427625 non-null  float64
 2   label             427625 non-null  int64  
 3   rec/sent          427625 non-null  int64  
 4   amount            427625 non-null  float64
 5   block_id          427625 non-null  int64  
 6   size              427625 non-null  int64  
 7   weight            427625 non-null  int64  
 8   version           427625 non-null  int64  
 9   lock_time         427625 non-null  int64  
 10  is_coinbase       427625 non-null  int64  
 11  has_witness       427625 non-null  int64  
 12  input_count       427625 non-null  int64  
 13  output_count      427625 non-null  int64  
 14  input_total       427625 non-null  int64  
 15  input_total_usd   427625 non-null  float64
 16  output_total      42

In [5]:
df['label'].value_counts()

0    335847
2     55390
3     34134
1      2254
Name: label, dtype: int64

In [5]:
import warnings 
warnings.filterwarnings(action= 'ignore')

In [6]:
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef,classification_report,roc_curve
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [7]:
def f1():
  X=df.drop('label',axis=1)
  y=df['label']
  return X,y
X,y=f1()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
def f2():
  X,y=f1()
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
  norm = MinMaxScaler().fit(X_train)
  X_train = norm.transform(X_train)
  X_test = norm.transform(X_test)
  sm = SMOTE(random_state = 2)
  X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
  return X_train,X_test,y_train,y_test,X_train_res,y_train_res
X_train,X_test,y_train,y_test,X_train_res,y_train_res=f2()
def f3():
  return X_train,X_test,y_train,y_test,X_train_res,y_train_res;

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
def cls1():
  X_train,X_test,y_train,y_test,X_train_res,y_train_res=f3()
  dtc=DecisionTreeClassifier(criterion='entropy',max_depth=6)
  ovr=OneVsRestClassifier(dtc)
  ovr.fit(X_train,y_train)
  return ovr.predict_proba(X_test),y_test
pp,y_test=cls1()
fpr = {}
tpr = {}
thresh ={}
n_class = 4
X_train,X_test,y_train,y_test,X_train_res,y_train_res=f3()
for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, pp[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='red', label='Exchange vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Servies vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Pools vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--',color='brown', label='Gambling vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=600); 

In [13]:
clf = RandomForestClassifier()
clf.fit(X,y)
scores = cross_val_score(clf, X, y, cv=3,scoring='accuracy')
print("Accuracy:"+ str(scores.mean()))

Accuracy:0.9873534888189611


**1. Grid Search**

In [14]:
#Random Forest
from sklearn.model_selection import GridSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': [10, 20, 30],
    #'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    #'min_samples_leaf': [1,2,4,8],
    #"bootstrap":[True,False],
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
grid = GridSearchCV(clf, rf_params, cv=3, scoring='accuracy')
grid.fit(X, y)
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))

{'criterion': 'entropy', 'max_depth': 15, 'n_estimators': 10}
Accuracy:0.9669383060773787


**2. Random Search**

In [15]:
#Random Forest
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,64),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['gini','entropy']
}
n_iter_search=20 #number of iterations is set to 20, you can increase this number if time permits
clf = RandomForestClassifier(random_state=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='accuracy')
Random.fit(X, y)
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))

{'criterion': 'entropy', 'max_depth': 8, 'max_features': 12, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 90}
Accuracy:0.9596047654062708


In [21]:
!pip install git+https://github.com/scikit-optimize/scikit-optimize.git

  Cloning https://github.com/scikit-optimize/scikit-optimize.git to /tmp/pip-req-build-xs6r57cr
  Running command git clone -q https://github.com/scikit-optimize/scikit-optimize.git /tmp/pip-req-build-xs6r57cr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-optimize: filename=scikit_optimize-0.9.dev0-cp37-none-any.whl size=102003 sha256=dfb4c6a00bb13a0a5d49b61515d42a25854e6154084e77801089d279f143ea39
  Stored in directory: /tmp/pip-ephem-wheel-cache-nla6t2iw/wheels/11/6f/86/2b772172db85ad0b4487d67e325e535ee8e7782b2a1dfcadf5
Successfully built scikit-optimize
  Found existing installation: scikit-optimize 0.8.1
    Uninstalling scikit-optimize-0.8.1:
      Successfully uninstalled scikit-optimize-0.8.1


In [41]:
! pip install git+https://github.com/thuijskens/scikit-hyperband.git

  Cloning https://github.com/thuijskens/scikit-hyperband.git to /tmp/pip-req-build-eoafm0mp
  Running command git clone -q https://github.com/thuijskens/scikit-hyperband.git /tmp/pip-req-build-eoafm0mp
     |████████████████████████████████| 163kB 6.9MB/s 
  Created wheel for scikit-hyperband: filename=scikit_hyperband-0.0.1-cp37-none-any.whl size=10362 sha256=2021e31d1cb2a0cdb67fe2dff89b7728d47e765fad2cf535ea2951b4ad659be3
  Stored in directory: /tmp/pip-ephem-wheel-cache-qbgsgw1w/wheels/9f/b0/a4/090ef56209908e0e7d583e47d3935f14593d31d9491b93cd3e
Successfully built scikit-hyperband


**3.Hyper Band**

In [42]:
#Random Forest
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,23),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=10,max_iter=100,scoring='accuracy')
hyper.fit(X, y)
print(hyper.best_params_)
print("Accuracy:"+ str(hyper.best_score_))

{'criterion': 'gini', 'max_depth': 20, 'max_features': 4, 'min_samples_leaf': 9, 'min_samples_split': 5, 'n_estimators': 11}
Accuracy:0.9717135340543701


**4. BO-GP**

Using BayseSearch_CV

In [25]:
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': Integer(10,100),
    "max_features": Integer(1,23),
    'max_depth': Integer(5,50),
    "min_samples_split":Integer(2,11),
    "min_samples_leaf":Integer(1,11),
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, n_jobs=-1,scoring='accuracy')
#number of iterations is set to 20, you can increase this number if time permits
Bayes.fit(X, y)
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("Accuracy:"+ str(Bayes.best_score_))

OrderedDict([('criterion', 'entropy'), ('max_depth', 5), ('max_features', 23), ('min_samples_leaf', 11), ('min_samples_split', 2), ('n_estimators', 10)])
Accuracy:0.9790002923121894


Using skopt.gp_minimize

In [26]:
#Random Forest
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = RandomForestClassifier()
# Define the hyperparameter configuration space
space  = [Integer(10, 100, name='n_estimators'),
            Integer(5, 50, name='max_depth'),
          Integer(1, 23, name='max_features'),
          Integer(2, 11, name='min_samples_split'),
          Integer(1, 11, name='min_samples_leaf'),
         Categorical(['gini', 'entropy'], name='criterion'),]
# Define the objective function
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="accuracy"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("Accuracy:%.4f" % -res_gp.fun)
print(res_gp.x)

Accuracy:0.9780
[10, 5, 17, 11, 1, 'entropy']


In [ ]:
!pip install  git+https://github.com/claesenm/optunity.git
#!echo "export PYTHONPATH=$PYTHONPATH:$(pwd)/optunity" >> ~/.bashrc


In [52]:
# pip install optunity

In [59]:
# ! cd  usr/local/lib/python3.7/dist-packages/

/bin/bash: line 0: cd: usr/local/lib/python3.7/dist-packages/: No such file or directory


**5.PSO**

In [11]:
#Random Forest
import optunity
import optunity.metrics

data=X
labels=y.tolist()
# Define the hyperparameter configuration space
search = {
    'n_estimators': [10, 100],
    'max_features': [1, 23],
    'max_depth': [5,50],
    "min_samples_split":[2,11],
    "min_samples_leaf":[1,11],
    "criterion":[0,1]
         }
# Define the objective function
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_estimators=None, max_features=None,max_depth=None,min_samples_split=None,min_samples_leaf=None,criterion=None):
    # fit the model
    if criterion<0.5:
        cri='gini'
    else:
        cri='entropy'
    model = RandomForestClassifier(n_estimators=int(n_estimators),
                                   max_features=int(max_features),
                                   max_depth=int(max_depth),
                                   min_samples_split=int(min_samples_split),
                                   min_samples_leaf=int(min_samples_leaf),
                                   criterion=cri,
                                  )
    #predictions = model.predict(x_test)
    scores=np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="accuracy"))
    #return optunity.metrics.roc_auc(y_test, predictions, positive=True)
    return scores#optunity.metrics.accuracy(y_test, predictions)

optimal_configuration, info, _ = optunity.maximize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("Accuracy:"+ str(info.optimum))

KeyError: ignored

**6. GA**

In [12]:
pip install git+https://github.com/DEAP/deap@master

  Cloning https://github.com/DEAP/deap (to revision master) to /tmp/pip-req-build-m7hlcxp9
  Running command git clone -q https://github.com/DEAP/deap /tmp/pip-req-build-m7hlcxp9
  Created wheel for deap: filename=deap-1.3.1-cp37-cp37m-linux_x86_64.whl size=138882 sha256=dee2629a9ff81e0191588a1b8ccb87fb63db65837416ce14265d54b25b7e22ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-drfd3a0j/wheels/41/5d/79/113af333e80b363bbbbfd90549b0d2bcdcc05e644b5c77c3c6
Successfully built deap


In [13]:
#Random Forest
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': np.logspace(1,1.8,num = 10 ,base=20,dtype='int'),
    'max_depth': np.logspace(1,2,num = 10 ,base=10,dtype='int'),
    "max_features":np.logspace(0.2,1,num = 5 ,base=8,dtype='int'),
    "min_samples_split":np.logspace(0.4, 1, num=5, base=10, dtype='int'), #[2, 3, 5, 7, 10],
    "min_samples_leaf":np.logspace(0.1,1,num = 5 ,base=11,dtype='int'),
    "criterion":['gini','entropy']
}
rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,64),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #Categorical(name='criterion', categories=['gini','entropy'])#
    "criterion":['gini','entropy']
}
clf = RandomForestClassifier(random_state=0)
# Set the hyperparameters of GA 
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))

ModuleNotFoundError: ignored